# Minimum Eigen Optimizer
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

## Introduction

An interesting class of optimization problems to be addressed by quantum computing are Quadratic Unconstrained Binary Optimization (QUBO) problems.
QUBO is equivalent to finding the ground state of a corresponding Hamiltonian, which is an important problem not only in optimization, but also in quantum chemistry and physics. For this translation, the binary variables taking values in $\{0, 1\}$ are replaced by spin variables taking values in $\{-1, +1\}$, which allows to replace the resulting spin variables by Pauli Z matrices, and thus, an Hamiltonian. For more details on this mapping we refere to [1].

Qiskit Optimization provides automatic conversion from a suitable `QuadraticProgram` to an Ising Hamiltonian, which then allows to leverage all the `MinimumEigenSolver` available in Qiskit Aqua, such as
- `VQE`,
- `QAOA`, or
- `NumpyMinimumEigensolver` (classical exact method).

Qiskit Optimization wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called `Operator`), the call to an `MinimumEigensolver` as well as the translation of the results back to `OptimizationResult` in the `MinimumEigenOptimizer`.

In the following we first illustrate the conversion from a `QuadraticProgram` to an `Operator` and then show how to use the `MinimumEigenOptimizer` with different `MinimumEigensolver` to solve a given `QuadraticProgram`.
The algorithms in Qiskit Optimization automatically try to convert a given problem to the supported problem class if possible, for instance, the `MinimumEigenOptimizer` will automatically translate integer variables to binary variables or add a linear equality constraints as a quadratic penalty term to the objective.

### References
[1] [A. Lucas, *Ising formulations of many NP problems,* Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

## Converting a QUBO to an Operator

In [1]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToOperator

In [2]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



Next we translate this QUBO into an Ising operator. This results not only in an `Operator` but also in a constant offset to be taking into account to shift the resulting value.

In [3]:
qp2op = QuadraticProgramToOperator()
op, offset = qp2op.encode(qubo)
print('offset: {}'.format(offset))
print('operator:')
print(op.print_details())

offset: 1.5
operator:
IIZ	(-0.5+0j)
IZI	(0.25+0j)
ZII	(-1.75+0j)
IZZ	(0.25+0j)
ZIZ	(-0.25+0j)
ZZI	(0.5+0j)



## Solving a QUBO with the `MinimumEigenOptimizer`

We start by initializing the `MinimumEigensolver` we want to use.

In [4]:
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()

Then, we use the `MinimumEigensolver` to create `MinimumEigenOptimizer`.

In [5]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

We first use the `MinimumEigenOptimizer` based on the classical exact `NumPyMinimumEigensolver` to get the optimal benchmark solution for this small example.

In [6]:
exact_result = exact.solve(qubo)
print(exact_result)

([0.0,1.0,0.0] / -2.0)


Next we apply the `MinimumEigenOptimizer` based on `QAOA` to the same problem.

In [7]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

([0.0,1.0,0.0] / -2.0)


## `RecursiveMinimumEigenOptimizer`

It is conjectured that the circuit depth for `QAOA` has to be increased with the problem size, which might be prohibitive for near-term quantum devices.
A potential workaround is Recursive QAOA, as introduced in [2].
Qiskit Optimization generalizes this concept to the `RecursiveMinimumEigenOptimizer`.
The `RecursiveMinimumEigenOptimizer` takes an `MinimumEigenOptimizer` as input and applies the recursive optimization scheme to reduce the size of the problem one variable at a time.
Once the size of the generated intermediate problem is below a given threshold (`min...`), the `RecurisveMinimumEigenOptimizer` uses another solver (`...`), e.g., an exact classical solver such as CPLEX or the `MinimumEigenOptimizer` based on the `NumPyMinimumEigensolver`.

In the following, we show how to use the `RecursiveMinimumEigenOptimizer` using the two `MinimumEigenOptimizer` introduced before.

### References
[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, *Obstacles to State Preparation and Variational Optimization from Symmetry Protection,* arXiv  preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)

First, we construct the `RecursiveMinimumEigenOptimizer` such that it reduces the problem size from 3 variables to 1 variable and then uses the exact solver for the last variable. Then we call `solve` to optimize the considered problem.

In [8]:
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

In [9]:
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

([0.0,1.0,0.0] / -2.0)
